# 6.6. Чисельне розв'язування мішаних задач для параболічних рівнянь методом скінчених різниць
---------------------

## 6.6.1.  Постановка мішаної задачі для рівняння теплопровідності 

Нехай 
$Q:=(0,l)\times (0,T]$, $\overline{Q}=[0,l]\times[0,T]$, де $l>0$, $T>0$ -- деякі числа.

Розглянемо мішану задачу першого роду для рівняння теплопровідності: знайти функцію $u\in C^{2,1}(Q)\cap C(\overline{Q})$,
яка задовольняє рівняння:

$(1)\quad\qquad u_t-a^2u_{xx}=f(x,t),\quad (x,t)\in Q,$

крайові умови:

$(2)\quad\qquad u|_{x=0}=0, \quad  u|_{x=l}=0,\quad t\in (0,T],$

та початкову умову

$(3)\quad\qquad u|_{t=0}=\varphi(x),\quad x\in[0,l],$

де  $a>0$ -- деяка стала, $f\in C(\overline{Q})$, $\varphi\in C([0,l])$ -- задані функції, причому виконується умова узгодження: $\varphi(0)=\varphi(l)=0$.

## 6.4.2.  Різницева схема 

Нехай $N_x, N_t$ -- які-небудь натуральні числа і
$$
h:=\frac{l}{N_x},\quad \tau:=\frac{T}{N_t}.
$$
Побудуємо сітку
$$
\{(x_i,t_j) \;|\; x_i:=ih,\,\,t_j:=j\tau, \quad i=\overline{0,N_x}, \,\,j=\overline{0,N_t}\},
$$
на якій розглянемо наближення функції $u$
$$
u_{i,j}\approx u(x_i,t_j),\,\,i=\overline{0,N_x},\,\,j=\overline{0,N_t},
$$
і сіткові значення вхідних даних: 
$$f_{i,j}:=f(x_i,t_j), \quad i=\overline{1,N_x-1},\; j=\overline{1,N_t-1}, \quad \varphi_{i}:=\varphi(x_i),\;\;i=\overline{0,N_x}. $$


Якщо у внутрішніх вузлах сітки використати такі апроксимації похідних функції $u$:
$$u_{xx} (x_i,t_j) \approx \frac{u_{i-1,j}-2u_{i,j}+u_{i+1,j}}{h^2}, \quad
i=\overline{1,N_x-1}, \,\,j=\overline{0,N_t-1}, $$
$$u_{t} (x_i,t_j) \approx \frac{u_{i,j+1}-u_{i,j}}{\tau}, \quad i=\overline{1,N_x-1},
\,\,j=\overline{0,N_t-1}.$$

то, ввівши позначення  $\alpha:=\dfrac{\tau^2}{h^2}a^2$, у результаті отримаємо явну різницеву схему відносно шуканих наближень розв'язку:

$(4)\quad\qquad
  u_{i,j+1}=\alpha(u_{i-1,j}+u_{i+1,j})+
  \big(1-2\alpha)u_{i,j} + \tau f_{i,j}, \quad   i=\overline{1,N_x-1}, \quad j=\overline{0,N_t-1}.$
 
$(5)\quad\qquad\qquad u_{0,j}=0, \quad u_{N_x,j}=0,\quad j=\overline{1,N_t},$

$(6)\quad\qquad\qquad u_{i,0}=\varphi_i,\quad i=\overline{0,N_x}.$

Обчислення за нею проводять в такому порядку: спочатку  за формулами (6) і (5)
задаємо $u_{i,0}$, $i=\overline{0,N_x}$, і $u_{0,j}, \;\; u_{N_x,j},\quad j=\overline{1,N_t}$.
Далі за формулою (4) при $j=0$  знаходимо $u_{i,1}$, $i=\overline{1,N_x-1}$, а потім за цією ж формулою при $j=1$ знаходимо   $u_{i,2}$, $i=\overline{1,N_x-1}$, і так продовжуємо включно до $j=N_t-1$, тобто матимемо завершальне значення $u_{i,N_t}$, $i=\overline{1,N_x-1}$.

#### Пояснення до використання програмного коду
-----------------
*   Підготувати потрібні функції : 
    1.   виконати комірку для підготовки середовища 
    2.   виконати комірки, де **визначені** функції ``FDA_E_solver``,``set_matrix`` і ``set_vector``

*    Обчислити чисельний розв'язок конкретної крайової задачі :
     1.  виконати комірку, де **визначені** функції ``phi1``,``phi2``, ``psi1``, ``psi2``  і ``f``, які задають конкретну крайову задачу 
     2.  виконати комірку, в якій задано усі параметри крайової задачі 
     3.  виконати комірку з викликом функції ``FDA_E_solver``, перед виконанням задати відповідні аргументи цієї функції.

*   Щоб переконатися, що чисельний розв'язок достатньо точний, можна виконати кілька послідовних викликів функції ``FDA_E_solver``, збільшуючи кількість вузлів сітки.

#### Програмна реалізація методів
------------

>#### Підготовка середовища

In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import linalg
from mpl_toolkits import mplot3d

>#### ``FDA_P_solver`` -- функція, яка отримує вхідні параметри чисельного методу, формує масиви значень розв'язку на межі області, задає матрицю вектор вільних членів  СЛАР, знаходить значення чисельного розв'язку у внутрішніх вузлах сітки і повертає матрицю, яка містить значення розв'язку у всіх точках сітки. За замовчуванням ( при ``plotting=True``) будуються 3D-графіки чисельного розв'язку задачі.
    


In [46]:
def FDA_P_solver(f, phi, a, l, T, Nx, Nt, plotting=True):
    """ Розв'язування мішаної задачі (1)-(3) 
        методом скінчених різниць 
    """   
    h = l/Nx
    tau = T/Nt
    alpha =a*a*tau/(h*h)
    #print(alpha)      
    
    x=np.linspace(0, l, Nx+1)
    t=np.linspace(0, T, Nt+1)  
    #print(f"x={x}")
    #print(f"t={t}")
    
    fij = np.empty((Nt+1,Nx+1), dtype=float) 
    
    for j in range(Nt+1):
        fij[j,0]=0
        fij[j,Nx]=0
        for i in range(1,Nx): 
            fij[j,i] = f(x[i],t[j])
    #print(f"fij={fij}")
    
    if Nx < 3 :
        raise Exception('invalid grid')
        
    u = np.empty((Nt+1,Nx+1), dtype=float)
    
    for i in range(Nx+1):
        u[0,i] = phi(x[i])  
    
    if Nx < 3:
        for j in range(1,Nt+1):
            u[j,0] = 0
            u[j,Nx] = 0
            u[j,1] = alpha * u[j-1,2]+(1-2*alpha)*u[j-1,1]+tau*fij[j-1,1]
            u[j,2] = alpha * u[j-1,1]+(1-2*alpha)*u[j-1,2]+tau*fij[j-1,2]
    else:
        for j in range(1,Nt+1):
            u[j,0] = 0
            u[j,Nx] = 0            
            for i in range(1,Nx):             
                u[j,i] = alpha *(u[j-1,i-1] + u[j-1,i+1])+(1-2*alpha)*u[j-1,i]+tau*fij[j-1,i]
    
    
    if plotting :
        TT, X = np.meshgrid(t, x)
        # set up a figure twice as wide as it is tall
        fig = plt.figure(figsize=plt.figaspect(0.5)) 
        # =============
        # First subplot
        # =============
        # set up the axes for the first plot
        ax = fig.add_subplot(1, 2, 1, projection='3d')
        ax.set_xlabel('t')
        ax.set_ylabel('x') 
        ax.set_zlabel('u');
        ax.set_title(f"Розв'язок задачі  Nt={Nt}, Nx={Nx}");
        surf = ax.plot_surface(TT, X, u.T, rstride=1, cstride=1, cmap='viridis', linewidth=0, antialiased=False)
        fig.colorbar(surf, shrink=0.5, aspect=10)
        # ==============
        # Second subplot
        # ==============
        # set up the axes for the second plot
        ax = fig.add_subplot(1, 2, 2, projection='3d')
        ax.set_xlabel('t')
        ax.set_ylabel('x')
        ax.set_zlabel('u');
        ax.set_title(f"Розв'язок задачі  Nt={Nt}, Nx={Nx}");
        surf = ax.plot_wireframe(TT, X, u.T, rstride=1, cstride=1)
    return u

>#### ``phi``  і ``f`` -- функції, які задають конкретну мішану задачу 

#### Обчислювальні експерименти
------------

Продемонструємо застосування методу скінчених різниць до розв'язування мішаної задачі для рівняння теплопровідності.

**Приклад 1.** (приклад 1.7) Методом скінчених різниць розв'язати мішану задачу для рівняння теплопровідності
\begin{equation}\label{2001.6.17}
u_t- u_{xx}= x^2+t,\,\,(x,t)\in(0,3)\times(0,3],
\end{equation}
\begin{equation}\label{2001.6.18}
u|_{x=0}=0,\,\,u|_{x=3}=0,\quad t\in(0,3],
\end{equation}
\begin{equation}\label{2001.6.19}
u|_{t=0}=x(3-x),\quad x\in[0,3].
\end{equation}
застосовуючи метод скінчених різниць на послідовності сіток, в яких подвоюється кількість вузлів по кожній змінній. 

Визначимо функції, які задають праву частину рівняння (1) і початкову умову:

In [47]:
def f(x,t):
    return x*x+t
def phi(x):
    return x*(3-x)

Задамо дані області визначення розв'язку і початкові значення параметрів сітки:

In [50]:
a = 1
l = 3
T = 3

Nx_start = 6
Nt_start = 24

Знайдемо чисельні розв'язки задачі на послідовності сіток, подвоюючи на кожному кроці кількість вузлів по кожній змінній і зберігаючи відповідні масиви результатів; для подальшого аналізу також зберігатимемо значення чисельного розв'язку у точках початкової сітки:

In [51]:
Nx=Nx_start
Nt=Nt_start
u_0 = FDA_P_solver(f, phi, a, l, T, Nx, Nt)
u0=u_0[1:Nt:1,1:Nx:1].reshape((Nx-1)*(Nt-1))

C:\Users\AF9AD~1.MUZ\AppData\Local\Temp/ipykernel_13860/357893538.py:53: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=plt.figaspect(0.5))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
for j in range(Nt+1):
    print(f"j={j}")
    for i in range(1,Nx):             
        print(f"i={i}, u[i,j]={u_0[j,i]}")

j=0
i=1, u[i,j]=2.0
i=2, u[i,j]=2.0
j=1
i=1, u[i,j]=1.0
i=2, u[i,j]=4.0
j=2
i=1, u[i,j]=5.0
i=2, u[i,j]=2.0
j=3
i=1, u[i,j]=0.0
i=2, u[i,j]=9.0


In [14]:
Nx *=2
Nt *=2
u_1 = FDA_P_solver(f, phi, a, l, T, Nx, Nt)
u1=u_1[2:Nt-1:2,2:Nx-1:2].reshape((Nx-1)*(Nt-1))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: cannot reshape array of size 4 into shape (25,)

In [33]:
Nx=Nx_start*64
Nt=Nt_start*128
u_0 = FDA_H_solver(f, phi, psi, a, l, T, Nx, Nt)
u0=u_0[1:Nx:1,1:Nt:1].reshape((Nx-1)*(Nt-1))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: cannot reshape array of size 65280 into shape (130305,)

In [ ]:
Nx *= 2
Ny *= 2
u_1 = FDA_E_solver(f, phi1, phi2, psi1, psi2, a, b, Nx, Ny)
u1=u_1[2:Nx-1:2,2:Ny-1:2].reshape((Nx_start-1)*(Ny_start-1))

In [ ]:
Nx *= 2
Ny *= 2
u_2 = FDA_E_solver(f, phi1, phi2, psi1, psi2, a, b, Nx, Ny)
u2=u_2[4:Nx-3:4,4:Ny-3:4].reshape((Nx_start-1)*(Ny_start-1))

In [ ]:
Nx *= 2
Ny *= 2
u_3 = FDA_E_solver(f, phi1, phi2, psi1, psi2, a, b, Nx, Ny)
u3=u_3[8:Nx-7:8,8:Ny-7:8].reshape((Nx_start-1)*(Ny_start-1))

In [ ]:
Nx *= 2
Ny *= 2
u_4 = FDA_E_solver(f, phi1, phi2, psi1, psi2, a, b, Nx, Ny)
u4=u_4[16:Nx-15:16,16:Ny-15:16].reshape((Nx_start-1)*(Ny_start-1))

Як бачимо, графіки чисельних розв'язків візуально подібні між собою. Для детальнішого аналізу занесемо в таблицю значення усіх розв'язків на спільній для усіх сіток множині вузлів: 

In [ ]:
df=pd.DataFrame({'u_0':u0, 'u_1':u1,'u_2':u2, 'u_3':u3,'u_4':u4})
df

In [ ]:
df1=pd.DataFrame()
df1['e_0']=np.abs(df['u_4']-df['u_0'])
df1['e_1']=np.abs(df['u_4']-df['u_1'])
df1['e_2']=np.abs(df['u_4']-df['u_2'])
df1['e_3']=np.abs(df['u_4']-df['u_3'])

df1

In [ ]:
fig = plt.figure(figsize=(8, 5))
df1.e_0.plot(logy=True)
df1.e_1.plot(logy=True)
df1.e_2.plot(logy=True)
df1.e_3.plot(logy=True)

ax = fig.gca()
ax.legend();

In [ ]:
ne_0 = max(np.abs(df1['e_0']))
ne_1 = max(np.abs(df1['e_1']))
ne_2 = max(np.abs(df1['e_2']))
ne_3 = max(np.abs(df1['e_3']))

print(f" ne_0={ne_0:1.2e}\n ne_1={ne_1:1.2e}\n ne_2={ne_2:1.2e}\n ne_3={ne_3:1.2e}")

In [ ]:
df2=pd.DataFrame()
df2['r_0'] = df1['e_0'] / df1['e_1']
df2['r_1'] = df1['e_1'] / df1['e_2']
df2['r_2'] = df1['e_2'] / df1['e_3']

df2

In [52]:
plt.close('all')